In [2]:
# from mp (preamble)

import pandas as pd
import numpy as np
import copy

from scipy.stats import chi2_contingency
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

demos = pd.read_csv("demos.csv")

In [3]:
demos['player_loadout_array'] = demos['player_loadout'].str.split(', ')
unique_items = demos['player_loadout_array'].explode().unique().tolist()
knives = [i for i in unique_items if
          "knife" in i.lower() or
          "daggers" in i.lower() or
          "bayonet" in i.lower() or
          "karambit" in i.lower()]

knives

['Butterfly Knife',
 'Talon Knife',
 'M9 Bayonet',
 'Karambit',
 'Shadow Daggers',
 'Skeleton Knife',
 'Flip Knife',
 'Bayonet',
 'Survival Knife',
 'Nomad Knife',
 'Stiletto Knife',
 'knife_t',
 'knife',
 'Ursus Knife']

In [4]:
demos['player_loadout_array'] = demos['player_loadout_array'].apply(lambda l: [i for i in l if i not in knives])
demos['player_loadout_array'].head(10)


0              [Glock-18, C4 Explosive]
1    [Glock-18, Smoke Grenade, Molotov]
2                            [Glock-18]
3      [P250, Smoke Grenade, Flashbang]
4                               [USP-S]
5                [P2000, Smoke Grenade]
6                       [Dual Berettas]
7                               [USP-S]
8                    [USP-S, Flashbang]
9                            [Glock-18]
Name: player_loadout_array, dtype: object

### Analyzing How Common Groups of Weapons Affect Winrate

For the inventory data, we wanted to see how common sets of weapons (guns) that a team bought in a round affected their probability of winning. In order to do this, we used Association Rule Mining to determine these rules followed by Chi-Square tests to determine if the presence of each mined rule has a signifiant correlation with round win. For this test, each round was converted to a basket, then we used a modified association rule mining algorithm that factors in quantities of items.

#### The Intuition

An input basket to this algorithm will look like the following: $B = \{AK_1, AK_2, AK_3, M4_1, M4_2, win\}$

Each subscript refers to the $nth$ occurence of an item.

Given a mined rule $I_n$ where $I$ is an item and $n$ is the $nth$ occurence of the item, the rule can be interpreted as "the itemset contained at least $n$ of $I$". 

With this method, the above basket $B$ contains "at least 1, 2 and 3 AK and at least 1 and 2 M4". 

Since we are using "at least", any basket with containing $I_x$ will also contain $I_{x-1}$ if $x \geq 2$. Due to this, any item $I_{n+1}$ will increase the support of any itemset cotaining $I_n$.

Since there may be generated rules that contain duplicates such as $\{ AK_1, AK_2, M4_2 \} \rightarrow \{ win \}$, only the **highest** one is kept, since having a higher minimum quantity implies the lower one.

#### The General Process

Our plan for the association rule mining of inventories is as follows:

1. Perform additional data cleaning, such as by ignoring empty inventories caused by players dropping their gun when the inventories were recorded
2. Determine the "best" weapon of a player (primary weapon first, fallback secondary - more on this later)
3. Create baskets with the 5 guns of each team per round, and an additional "win" or "loss" item for round result
4. Mark duplicate guns (i.e. $\{AK, AK, AK, M4, M4, win\}$ becomes $\{AK_1, AK_2, AK_3, M4_1, M4_2, win\}$)
5. Perform Association Rule Mining using the Apriori algorithm on the generated baskets
6. Filter rules to only include those that have a consequent of "win" or "loss"
7. Keep only "highest" quantities of each rule and discard duplicates
8. Perform Chi-Square tests on the final rules to confirm statistical significance of the correlation between the presence of the rule and the round/result

#### Removing Rounds with Empty Inventories

Some players had no guns in their inventory at the tick that inventories were recorded. Since these values are just outliars, we decided to ignore these rounds which these incomplete inventories to avoid any strange results. Since the data would still be useful for other tests, we only ignore them for the inventory association rule mining.

In [5]:
outliar_rounds = demos.loc[demos['player_loadout_array'].str.len() == 0]['round_id'].values

demos_inventory = demos.loc[demos['round_id'].isin(outliar_rounds) == False]

#### Determining the Player's Weapon

In Counter-Strike, guns can be either "primary" (for guns including, but not limited to rifles, sub-machine guns and shotguns), or "secondary" (mostly for pistols). Each player can carry up to only one gun of each type. For the sake of analysis, we counted the primary weapon of each player, and fallback to their secondary gun if they do not have a primary gun.

To do this filtering, we first list down all items seen in the dataset:

In [6]:
item_types = demos_inventory['player_loadout_array'].explode().unique()
item_types

array(['Glock-18', 'C4 Explosive', 'Smoke Grenade', 'Molotov', 'P250',
       'Flashbang', 'USP-S', 'P2000', 'Dual Berettas', 'Tec-9',
       'Desert Eagle', 'MP9', 'Incendiary Grenade',
       'High Explosive Grenade', 'FAMAS', 'AK-47', 'M4A1-S', 'M4A4',
       'AWP', 'Galil AR', 'Five-SeveN', 'Decoy Grenade', 'MAC-10', 'AUG',
       'Zeus x27', 'SSG 08', 'XM1014', 'MAG-7', 'SG 553', 'CZ75-Auto',
       'MP7', 'MP5-SD', 'UMP-45'], dtype=object)

Then, we manually sorted them out as either "primary" or "secondary" gun based on the in-game classification:

In [7]:
secondaries = ['Glock-18', 'P250', 'USP-S', 'P2000', 'Dual Berettas', 'Tec-9',
       'Desert Eagle', 'CZ75-Auto', 'Five-SeveN']

primaries = ['MP9', 'FAMAS', 'AK-47', 'M4A1-S', 'M4A4',
       'AWP', 'Galil AR', 'MAC-10', 'AUG',
       'SSG 08', 'XM1014', 'MAG-7', 'SG 553', 
       'MP7', 'MP5-SD', 'UMP-45']

primary_ser = demos_inventory['player_loadout_array'].apply(lambda x: list(filter(lambda y: y is not None, [(item if item in primaries else None) for item in x]))).apply(lambda x: x[0] if len(x) > 0 else None)
demos_inventory = demos_inventory.assign(primary_weapon=primary_ser)

secondary_ser = demos_inventory['player_loadout_array'].apply(lambda x: list(filter(lambda y: y is not None, [(item if item in secondaries else None) for item in x]))).apply(lambda x: x[0] if len(x) > 0 else None)
demos_inventory = demos_inventory.assign(secondary_weapon=secondary_ser)

weapon_ser = np.where(demos_inventory['primary_weapon'].notna(), demos_inventory['primary_weapon'], demos_inventory['secondary_weapon'])
demos_inventory = demos_inventory.assign(weapon=weapon_ser)


demos_inventory.loc[demos_inventory['weapon'].isna()]

,match_id,map_id,round_id,team_name,map_name,round_number,round_ct_team,round_first_site_hit,round_site_hit_time,round_bomb_plant_site,...,round_first_killer,round_first_death,player_headshots,player_upperbodyshots,player_stomachshots,player_legshots,player_loadout_array,primary_weapon,secondary_weapon,weapon
5694,13,29,569,Team Falcons,de_ancient,15,HEROIC,A,67.312500,A,...,False,False,0,0,0,0,[C4 Explosive],None,None,None
6114,13,31,611,Team Falcons,de_dust2,6,HEROIC,A,51.781250,NaN,...,False,False,0,0,0,0,"[Smoke Grenade, Flashbang]",None,None,None
6559,14,33,655,Lynn Vision Gaming,de_anubis,11,FURIA,B,34.828125,B,...,False,False,0,1,0,0,[C4 Explosive],None,None,None
8476,18,43,847,FURIA,de_nuke,12,FURIA,B,34.359375,A,...,False,False,0,0,0,0,"[Flashbang, Smoke Grenade]",None,None,None
9260,20,47,926,Team Falcons,de_dust2,9,MOUZ,B,86.031250,B,...,False,False,0,0,0,0,[Flashbang],None,None,None
12249,25,61,1224,GamerLegion,de_nuke,18,Team Falcons,NaN,NaN,A,...,False,False,0,10,2,0,[Smoke Grenade],None,None,None
13375,27,67,1337,Team Vitality,de_inferno,1,Team Vitality,A,44.703125,NaN,...,False,False,0,0,0,0,[Flashbang],None,None,None


#### Removing additional outliar rounds

Upon attempting to get the player's weapon, we saw that some rounds still had no weapon. We realized that the original filtering only got rid of empty inventories and not inventories that contain some non-gun items such as grenades. Because these rounds still don't have weapons, we had to get rid of these rounds. 

In [8]:
demos_inventory.loc[demos_inventory['weapon'].isna()]['round_id'].values

array([ 569,  611,  655,  847,  926, 1224, 1337])

Getting rid of the rounds:

In [9]:
more_outliar_rounds = demos_inventory.loc[demos_inventory['weapon'].isna()]['round_id'].values
demos_inventory = demos_inventory.loc[demos_inventory['round_id'].isin(more_outliar_rounds) == False]
demos_inventory.loc[demos_inventory['weapon'].isna()]['round_id'].values

array([], dtype=int64)

#### Adding a "Result" Column

We add a result column in order to make adding $win$ or $loss$ to baskets easier later on

In [10]:
result = (demos_inventory['round_result'] == 'CT') & (demos_inventory['round_ct_team'] == demos_inventory['team_name']) | (demos_inventory['round_result'] == 'T') & (demos_inventory['round_ct_team'] != demos_inventory['team_name'])
demos_inventory = demos_inventory.assign(result=result) # makes it easier for us to do ARM
demos_inventory

,match_id,map_id,round_id,team_name,map_name,round_number,round_ct_team,round_first_site_hit,round_site_hit_time,round_bomb_plant_site,...,round_first_death,player_headshots,player_upperbodyshots,player_stomachshots,player_legshots,player_loadout_array,primary_weapon,secondary_weapon,weapon,result
0,0,0,0,Team Liquid,de_ancient,1,FaZe Clan,A,23.078125,NaN,...,False,1,1,1,0,"[Glock-18, C4 Explosive]",None,Glock-18,Glock-18,False
1,0,0,0,Team Liquid,de_ancient,1,FaZe Clan,A,23.078125,NaN,...,False,1,3,2,0,"[Glock-18, Smoke Grenade, Molotov]",None,Glock-18,Glock-18,False
2,0,0,0,Team Liquid,de_ancient,1,FaZe Clan,A,23.078125,NaN,...,False,1,1,0,0,[Glock-18],None,Glock-18,Glock-18,False
3,0,0,0,Team Liquid,de_ancient,1,FaZe Clan,A,23.078125,NaN,...,False,0,0,0,0,"[P250, Smoke Grenade, Flashbang]",None,P250,P250,False
4,0,0,0,FaZe Clan,de_ancient,1,FaZe Clan,A,23.078125,NaN,...,False,1,5,0,0,[USP-S],None,USP-S,USP-S,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14275,28,70,1427,MOUZ,de_inferno,21,Team Vitality,NaN,NaN,NaN,...,False,0,0,0,0,"[Glock-18, Galil AR, Smoke Grenade, Molotov, F...",Galil AR,Glock-18,Galil AR,False
14276,28,70,1427,Team Vitality,de_inferno,21,Team Vitality,NaN,NaN,NaN,...,False,1,1,0,0,"[Smoke Grenade, AK-47, Flashbang, Incendiary G...",AK-47,Five-SeveN,AK-47,True
14277,28,70,1427,Team Vitality,de_inferno,21,Team Vitality,NaN,NaN,NaN,...,False,0,5,1,1,"[Five-SeveN, AK-47, Smoke Grenade, High Explos...",AK-47,Five-SeveN,AK-47,True
14278,28,70,1427,MOUZ,de_inferno,21,Team Vitality,NaN,NaN,NaN,...,False,0,0,0,0,"[Glock-18, AK-47, Molotov, High Explosive Gren...",AK-47,Glock-18,AK-47,False


#### Generating Baskets

We first create a function is used to generate baskets by obtaining an aggregate of each team per round and adding "win" or "loss" depending on the round's result

In [11]:
def create_baskets_df(demos_inventory: pd.DataFrame):
    baskets_df = demos_inventory.groupby(
        ['round_id', 'team_name']
        )['weapon'].apply(
            lambda x : [a for a in x] + ["win" if demos_inventory.loc[
                (demos_inventory['round_id'] == x.name[0]) & (demos_inventory['team_name'] == x.name[1])
            ]['result'].iloc[0] == True else "loss"]
        ).reset_index().rename(columns={"weapon": "basket"})
    baskets_df = pd.merge(baskets_df, demos_inventory, how="inner", on=["team_name", "round_id"])[['round_id', 'team_name', 'basket', 'result']]
    baskets_df = baskets_df.drop_duplicates(subset=['round_id', 'team_name']).reset_index()
    baskets_df = baskets_df.assign(result=baskets_df['result'].apply(lambda x: "win" if x else "loss"))
    return baskets_df

#### Marking Duplicate Guns

The next function takes each basket and marks duplicate items using a map (Python object)

In [12]:
def mark_duplicates(basket: list):
    new_list = []
    counts = {}
    basket_clone = copy.deepcopy(basket)
    while len(basket_clone) != 0:
        item = basket_clone.pop(0)
        if item not in new_list:
            counts[item] = 1 if item not in counts else counts[item] + 1
            new_list += [item + "_" + str(counts[item]) if item != 'win' and item != 'loss' else item]
        else:
            new_list += [item + "_1" if item != 'win' and item != 'loss' else item]
    return new_list


This function then uses the above function and marks all of duplicates in all baskets in a dataframe

In [13]:
def mark_duplicates_in_df(baskets_df: pd.DataFrame):
    baskets_df = baskets_df.assign(marked_basket=baskets_df['basket'].apply(mark_duplicates))
    return baskets_df

#### Running Apriori Association Rule Mining

After getting a dataframe of marked baskets, we then run the standard Apriori algorithm for Association Rule Mining. This function also takes care of filtering to only rules with "win" or "loss" as a consequent. The minimum support in the library we are using, `mlxtend`, requires a support as a fraction of $\frac{min\ amount}{total\ items}$, so we needed to perform this calculation to get the minimum support 

In [14]:
def generate_rules_df(baskets_df: pd.DataFrame, min_rounds: int):
    te = TransactionEncoder()
    te_ary = te.fit(baskets_df['marked_basket']).transform(baskets_df['marked_basket'])
    basket_df = pd.DataFrame(te_ary, columns=te.columns_)

    min_support = min_rounds / (len(baskets_df) / 2)

    frequent_itemsets = apriori(basket_df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

    # Filtering for rules where 'win' or 'loss' is the consequence
    winloss_rules = rules[(rules['consequents'] == {'loss'}) | (rules['consequents'] == {'win'})]
    return winloss_rules

#### Keeping Only the Highest Sets

The `extract_highest` function takes a set and extracts the highest occurence of an item.

The `process_winloss_df` function applies the `extract_highest` function and removes the duplicate rules. Since higher quantity "items" imply all lower quantities, the supports of all duplicates are the same and removing duplicates is a "safe" operation.

In [15]:
def extract_highest(set: frozenset[str]):
    outset = {}
    for item in set:
        split = item.split('_')
        if split[0] not in outset or outset[split[0]] < split[1]:
            outset[split[0]] = split[1]
    items = []
    for item in outset:
        items += [item + "_" + str(outset[item])]
    return frozenset(items)

def process_winloss_df(winloss_rules: pd.DataFrame):
    winloss_rules = winloss_rules.assign(antecedents=winloss_rules['antecedents'].apply(extract_highest))
    winloss_rules = winloss_rules.drop_duplicates()
    winloss_rules = winloss_rules.reset_index().drop(columns=['index'])
    winloss_rules = winloss_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
    return winloss_rules

#### Performing Chi-Square Tests 

The functions defined below get the counts for the chi-square contingency table

In [16]:
def baskets_with_rule(baskets_df: pd.DataFrame, rule: frozenset[str]):
    return baskets_df.loc[baskets_df['marked_basket'].apply(lambda x: [item in x for item in rule]).apply(all)]

def baskets_without_rule(baskets_df: pd.DataFrame, rule: frozenset[str]):
    return baskets_df.loc[baskets_df['marked_basket'].apply(lambda x: [item in x for item in rule]).apply(all).apply(lambda x : not x)]


def count_win_with(baskets_df: pd.DataFrame, rule: frozenset[str]):
    valid_baskets_df = baskets_with_rule(baskets_df, rule)
    return len(valid_baskets_df.loc[valid_baskets_df['result'] == 'win'])

def count_win_without(baskets_df: pd.DataFrame, rule: frozenset[str]):
    valid_baskets_df = baskets_without_rule(baskets_df, rule)
    return len(valid_baskets_df.loc[valid_baskets_df['result'] == 'win'])

def count_loss_with(baskets_df: pd.DataFrame, rule: frozenset[str]):
    valid_baskets_df = baskets_with_rule(baskets_df, rule)
    return len(valid_baskets_df.loc[valid_baskets_df['result'] == 'loss'])

def count_loss_without(baskets_df: pd.DataFrame, rule: frozenset[str]):
    valid_baskets_df = baskets_without_rule(baskets_df, rule)
    return len(valid_baskets_df.loc[valid_baskets_df['result'] == 'loss'])


def get_chi_square_counts(baskets_df: pd.DataFrame, rules_df: pd.DataFrame):
    rules_df = rules_df.assign(win_with=rules_df['antecedents'].apply(lambda x: count_win_with(baskets_df, x)))
    rules_df = rules_df.assign(win_without=rules_df['antecedents'].apply(lambda x: count_win_without(baskets_df, x)))
    rules_df = rules_df.assign(loss_with=rules_df['antecedents'].apply(lambda x: count_loss_with(baskets_df, x)))
    rules_df = rules_df.assign(loss_without=rules_df['antecedents'].apply(lambda x: count_loss_without(baskets_df, x)))

    return rules_df

The `get_chisquare` function actually constructs the table and performs the Chi-Square test on a row (rule).

The `get_chisquare_table` performs the Chi-Square test on each rule.

For the Chi-Square tests, the null hypothesis $H_0$ will be "The presence of the rule being tested and round win are independent"

The alternative hypothesis $H_a$ will be "The presence of the rule being tested and round win are dependent"

We will perform these tests with $\alpha = 0.05$

In [17]:
def get_chisquare(row: pd.Series):
    chi2_table = [[row['win_with'], row['win_without']], [row['loss_with'], row['loss_without']]]
    use_correction = (row['win_with'] < 10) or (row['win_without'] < 10) or (row['loss_with'] < 10) or (row['loss_without'] < 10)
    chi2 = chi2_contingency(chi2_table, correction=use_correction)
    return (chi2.statistic, chi2.pvalue)

def get_chisquare_table(rules_df: pd.DataFrame, alpha = 0.05):
    chi2 = rules_df.apply(get_chisquare, axis=1)
    rules_df = rules_df.assign(chi2=chi2.apply(lambda x: x[0]), pvalue=chi2.apply(lambda x: x[1]))
    rules_df = rules_df.assign(chi2_result=rules_df.apply(lambda x: "dependent" if x['pvalue'] <= alpha else "independent", axis=1))
    return rules_df
    


#### Putting it all together

In the end, we run all of the functions that we defined above and create a final table of rules, which are then filtered to only those which were determined to be correlated with winning rounds. For our findings for this part, we will get the five rules with the highest support for win and loss for the following categories:

1. Overall
2. Pistol Rounds
3. T Side (non-pistol rounds)
3. CT Side (non-pistol rounds)

In [24]:
def get_rules_chisquare_table(demos_df: pd.DataFrame, min_rounds, alpha):
    baskets_df = create_baskets_df(demos_df)
    baskets_df = mark_duplicates_in_df(baskets_df)

    rules_df = generate_rules_df(baskets_df, min_rounds)
    rules_df = process_winloss_df(rules_df)
    rules_df = get_chi_square_counts(baskets_df, rules_df)
    rules_df = get_chisquare_table(rules_df, alpha)

    rules_df = rules_df.loc[rules_df['chi2_result'] == 'dependent']
    return rules_df

#### Overall Rules

In [27]:
overall_rules = get_rules_chisquare_table(demos_inventory, 20, 0.05)

##### Wins

In [31]:
overall_rules.loc[overall_rules['consequents'] == {'win'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
66,"(M4A4_1, M4A1-S_1, AK-47_2)",(win),0.015526,0.785714,1.572538,44,1372,12,1406,18.700439,1.529473e-05,dependent
30,"(M4A4_1, AK-47_2)",(win),0.029993,0.745614,1.492281,85,1331,29,1389,28.744140,8.259949e-08,dependent
64,"(M4A4_1, AK-47_2, AWP_1)",(win),0.021524,0.701149,1.403289,61,1355,26,1392,14.577417,1.345170e-04,dependent
65,"(AK-47_2, MAC-10_1, Galil AR_1)",(win),0.015526,0.698413,1.397812,44,1372,19,1399,10.182310,1.417944e-03,dependent
28,"(M4A1-S_1, AK-47_2)",(win),0.062456,0.694118,1.389216,177,1239,78,1340,42.389313,7.479764e-11,dependent


##### Losses

In [32]:
overall_rules.loc[overall_rules['consequents'] == {'loss'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
38,"(Glock-18_2, Desert Eagle_1)",(loss),0.018349,0.945455,1.889576,3,1413,52,1366,42.650944,6.543285e-11,dependent
40,"(P250_1, Desert Eagle_1)",(loss),0.017996,0.944444,1.887557,3,1413,51,1367,41.634434,1.100380e-10,dependent
37,"(Glock-18_1, Desert Eagle_1)",(loss),0.029287,0.892473,1.783687,10,1406,83,1335,59.138803,1.469330e-14,dependent
36,"(Desert Eagle_1, Five-SeveN_2)",(loss),0.014820,0.875000,1.748766,6,1410,42,1376,25.910318,3.576524e-07,dependent
44,"(USP-S_1, Desert Eagle_1)",(loss),0.019054,0.870968,1.740707,8,1408,54,1364,33.326923,7.789672e-09,dependent


#### Pistol Rounds

In [35]:
pistol_round_rules = get_rules_chisquare_table(
    demos_inventory.loc[
        (demos_inventory['round_number'] == 1) 
        | (demos_inventory['round_number'] == 13)
        ], 2, 0.05)

##### Wins

In [38]:
pistol_round_rules.loc[pistol_round_rules['consequents'] == {'win'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
28,"(Dual Berettas_1, P2000_1, USP-S_2)",(win),0.069767,0.782609,1.577446,18,110,5,125,7.079834,0.007796,dependent
10,"(Dual Berettas_1, P2000_1)",(win),0.069767,0.782609,1.577446,18,110,5,125,7.079834,0.007796,dependent
27,"(Dual Berettas_1, USP-S_1, P2000_1)",(win),0.069767,0.782609,1.577446,18,110,5,125,7.079834,0.007796,dependent
0,(Dual Berettas_1),(win),0.224806,0.659091,1.328480,58,70,30,100,14.188557,0.000165,dependent
12,"(Dual Berettas_1, USP-S_2)",(win),0.224806,0.659091,1.328480,58,70,30,100,14.188557,0.000165,dependent


##### Losses

In [37]:
pistol_round_rules.loc[pistol_round_rules['consequents'] == {'loss'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
7,(P250_1),(loss),0.131783,0.708333,1.405769,14,114,34,96,9.861279,0.001688,dependent
20,"(Glock-18_2, P250_1)",(loss),0.131783,0.708333,1.405769,14,114,34,96,9.861279,0.001688,dependent
21,"(Glock-18_3, P250_1)",(loss),0.131783,0.708333,1.405769,14,114,34,96,9.861279,0.001688,dependent
19,"(Glock-18_1, P250_1)",(loss),0.131783,0.708333,1.405769,14,114,34,96,9.861279,0.001688,dependent
22,"(P250_1, Glock-18_4)",(loss),0.127907,0.702128,1.393453,14,114,33,97,9.035558,0.002648,dependent


#### Non-Pistol T Rounds

In [39]:
t_normal_round_rules = get_rules_chisquare_table(
    demos_inventory.loc[
        ((demos_inventory['round_number'] != 1) 
        | (demos_inventory['round_number'] != 13)) &
        (demos_inventory['round_ct_team'] != demos_inventory['team_name'])
        ], 10, 0.05)


##### Wins

In [42]:
t_normal_round_rules.loc[t_normal_round_rules['consequents'] == {'win'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
23,"(M4A1-S_1, AK-47_2)",(win),0.040932,0.763158,1.547060,58,641,18,700,23.397899,1.317223e-06,dependent
53,"(M4A1-S_1, AK-47_1, AWP_1)",(win),0.019760,0.756757,1.534083,28,671,9,709,9.495729,2.059507e-03,dependent
17,"(M4A1-S_1, AK-47_1)",(win),0.043754,0.756098,1.532747,62,637,20,698,24.049021,9.391399e-07,dependent
27,"(M4A1-S_1, AWP_1)",(win),0.020466,0.743590,1.507392,29,670,10,708,10.051349,1.522364e-03,dependent
59,"(M4A1-S_1, AK-47_2, AWP_1)",(win),0.018349,0.742857,1.505906,26,673,9,709,7.947348,4.815791e-03,dependent


##### Losses

In [41]:
t_normal_round_rules.loc[t_normal_round_rules['consequents'] == {'loss'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
32,"(Glock-18_4, Desert Eagle_1)",(loss),0.017643,1.000000,1.973538,0,699,25,693,22.807297,1.790845e-06,dependent
37,"(Glock-18_1, Desert Eagle_2)",(loss),0.023289,0.970588,1.915492,1,698,33,685,28.119119,1.140733e-07,dependent
31,"(Glock-18_3, Desert Eagle_1)",(loss),0.025406,0.947368,1.869667,2,697,36,682,28.550396,9.129114e-08,dependent
30,"(Glock-18_2, Desert Eagle_1)",(loss),0.036697,0.945455,1.865890,3,696,52,666,42.261218,7.986061e-11,dependent
33,"(P250_1, Desert Eagle_1)",(loss),0.019054,0.931034,1.837432,2,697,27,691,19.628846,9.403868e-06,dependent


#### Non-Pistol CT Rounds

In [45]:
ct_normal_round_rules = get_rules_chisquare_table(
    demos_inventory.loc[
        ((demos_inventory['round_number'] != 1) 
        | (demos_inventory['round_number'] != 13)) &
        (demos_inventory['round_ct_team'] == demos_inventory['team_name'])
        ], 10, 0.05)

##### Wins

In [48]:
ct_normal_round_rules.loc[ct_normal_round_rules['consequents'] == {'win'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
73,"(M4A4_1, M4A1-S_1, AK-47_2)",(win),0.026817,0.760000,1.501980,38,679,12,688,13.377227,0.000255,dependent
28,"(AUG_1, AWP_1)",(win),0.021171,0.750000,1.482218,30,687,10,690,9.803995,0.001741,dependent
74,"(M4A1-S_1, AUG_1, AWP_1)",(win),0.016937,0.750000,1.482218,24,693,8,692,6.831169,0.008958,dependent
3,(AUG_1),(win),0.026817,0.745098,1.472530,38,679,13,687,12.099544,0.000504,dependent
15,"(AK-47_1, AUG_1)",(win),0.014114,0.740741,1.463919,20,697,7,693,5.148108,0.023272,dependent


##### Losses

In [47]:
ct_normal_round_rules.loc[ct_normal_round_rules['consequents'] == {'loss'}].sort_values(by="confidence", ascending=False).head(5)

,antecedents,consequents,support,confidence,lift,win_with,win_without,loss_with,loss_without,chi2,pvalue,chi2_result
56,"(USP-S_1, Five-SeveN_2)",(loss),0.014820,1.000000,2.024286,0,717,21,679,19.827180,0.000008,dependent
39,"(P250_1, Desert Eagle_1)",(loss),0.016937,0.960000,1.943314,1,716,24,676,20.251635,0.000007,dependent
54,"(Five-SeveN_1, USP-S_2)",(loss),0.017643,0.925926,1.874339,2,715,25,675,18.818927,0.000014,dependent
51,"(Five-SeveN_1, P250_1)",(loss),0.016231,0.920000,1.862343,2,715,23,677,16.781940,0.000042,dependent
63,"(P250_1, USP-S_2)",(loss),0.014820,0.913043,1.848261,2,715,21,679,14.763896,0.000122,dependent
